# 1. Install Dependencies

In [4]:
# Import gradio for fast prototype display
import gradio as gr
# Import huggingface transformer to use models
from transformers import pipeline

In [6]:
# Use moviepy for seperating audio from input video file
from moviepy.editor import VideoFileClip

  if event.key is 'enter':



# 2. Load up Pipelie

### Create translation pipeline

In [7]:
translation_pipeline = pipeline('translation', model='Helsinki-NLP/opus-mt-ar-en')


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

  warnings.warn("Recommended: pip install sacremoses.")

Device set to use cpu


In [8]:
# Test how pipeline function
results = translation_pipeline('لا أستطيع مساعدتك لأنني مشغول.') 

In [9]:
# Display test result
results[0]['translation_text']

"I can't help you because I'm busy."

### Create audio extract to text pipeline

In [10]:
audio_to_text_pipeline = pipeline("automatic-speech-recognition", model="smerchi/Arabic-Morocco-Speech_To_Text") # Import pipeline that fetch text from audio

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cpu


# 3. Extracting Audio from Video

In [12]:
# Create an empty file for testing
mp3_file = "input_audio.mp3"

# Define extracting audio from video function
def extracting_audio_from_video(mp4_file):
 # Load the video file using VideoFileClip
    video_clip = VideoFileClip(mp4_file)
    # Extract the audio track from the video
    audio_clip = video_clip.audio
    # Specify the output audio file name (ensure you define or pass 'mp3_file')
    audio_clip.write_audiofile(mp3_file)
    # Close the audio clip to release resources
    audio_clip.close()

Q1: What is video_clip ?<br/>
A1: video_clip is used to represent a video file, allowing you to manipulate or extract its content. With VideoFileClip, I can read video files in formats like .mp4, .avi, etc. And to access or edit the video frames, duration, and other metadata. Also, extract the audio component of the video using the .audio attribute.<br/>
<br/>
Q2: Why do I need to close the audio clip?<br/>
A2: I need to close the audio clip to release resources (such as file handles or memory buffers) associated with the audio file.
If you don't close it explicitly, these resources may remain occupied until the Python garbage collector automatically cleans them up, which could lead to memory leaks or resource exhaustion in larger scripts.<br/>
<br/>
Q3: Can I return the audio clip to a variable after closing it, instead of writing it into an MP3 file?<br/>
No, you cannot return or use the audio clip after closing it. Once you call .close(), the audio clip object becomes invalid, and its associated data is cleared from memory.

In [14]:
extracting_audio_from_video("safaa_test1.mp4")

MoviePy - Writing audio in input_audio.mp3


MoviePy - Done.


# 4. Create Gradio Function and interface

In [15]:
# Create translate function using translation pipeline, and return the result
def translate_transformers(from_text):
  results = translation_pipeline(from_text)
  return results[0]['translation_text']

In [19]:
# Combine functions and prepare for Gradio 
def translate_app(mp4_file):
  input_audio = extracting_audio_from_video(mp4_file)
  input_text = audio_to_text_pipeline(mp3_file)
  output_text = translate_transformers(input_text['text'])
  return output_text

In [20]:
interface_test = gr.Interface(
    fn=translate_app, # Put the combined function here
    inputs=gr.Video(), # Set input as video
    outputs="text" # Set output as text
)

In [21]:
# Launch Gradio app
interface_test.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://afcfac212fc1cdfff3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
